In [ ]:
from flask import Flask, abort, request, jsonify
import cPickle as pickle
import numpy as np
import pandas as pd


In [ ]:
# read in fitted model and needed preprocessing data
modelFname = 'enet.pyobj'
dataFname = '/home/jovyan/work/data/autot4.7.csv'

inputs = pickle.load(open(modelFname, 'rb'))
model = inputs['model']
imputer = inputs['imputer']
scaler = inputs['scaler']
trainColumns = inputs['trainColumns']
factorVars = inputs['factorVars']
numericVars= inputs['numericVars']
dateVars = inputs['dateVars']

# remove target variable from numeric variables
numericVars.remove('matkamittarilukema')
numericVars.remove('mileagePerDay')

In [ ]:
def format_input_data(inputVector):
    if inputVector.has_key('max_date') and inputVector.has_key('kayttoonottopvm'):
        usageTime = pd.to_datetime(inputVector['max_date'])-pd.to_datetime(inputVector['kayttoonottopvm'])
        inputVector['usageDays'] = usageTime.days
    else:
        inputVector['usageDays'] = np.nan
    
    for varName in factorVars:
        dummyVariable = '{}_{}'.format(varName, inputVector.pop(varName, 'NA'))
        if dummyVariable in trainColumns:
            inputVector[dummyVariable] = 1
    x = pd.Series(index=trainColumns)

    for v in trainColumns:
        varName = v.split('_')[0]
        if varName in factorVars:
            x[v] = inputVector.get(v, 0)
        else:
            
            x[v] = inputVector.get(v)
    return x

def impute_scale_input_data(x):
    xtmp = imputer.transform(x[numericVars].values.reshape(1, -1))
    xtmp = scaler.transform(xtmp)
    x[numericVars] = xtmp[0]
    return x

def parse_args(args):
    parsed = {}
    for key, value in args.iteritems():
        if key in numericVars:
            parsed[key] = args.get(key, type=np.float, default=np.nan)
        else:
            parsed[key] = args.get(key)
    return parsed

In [ ]:
app = Flask(__name__)

@app.route('/mileage')
def mileage():
    parsed = parse_args(request.args)
    formatted = format_input_data(parsed)
    scaled = impute_scale_input_data(formatted)
    response = {
        'estimate': model.predict(scaled.values.reshape(1, -1))[0]
    } 
    return jsonify(response)

In [ ]:
app.run(host='0.0.0.0')